In [1]:
import os
import pdb

import numpy as np
import pandas as pd


import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt


#import plottools as pt
import remtools as rt
import scoreblock as sb
import eegplotter as ep


%load_ext autoreload

In [3]:

def dig_out_scores(s=None, trial=None, classifier='OVR', m='24h_8ch'):
    """quick hack to get a human and model scores for one trial"""

    # WWRW scoreTag and scoreType columns, then the vector of scores
    # scoreType (human, model, consensus, switch, etc)
    # scoreTag (h-con, h-GS, m-OVR-24h_8ch, etc)
    
    
    
    
    # narrow down to trial
    scbt = scb.keeprows(conditions=[('trial', trial)])

    # human scores
    df_keep = scbt.df[~scbt.df['scorer'].isna()].copy()
    scb_hum = sb.ScoreBlock(df=df_keep, index_cols=scbt.copy_index_cols())

    scoreType = ['human']*scb_hum.numrows
    scoreTag = [ 'hum-%s' % (row['scorer']) for _, row in scb_hum.df.iterrows()]
    scb_hum.df['scoreType'] = scoreType
    scb_hum.df['scoreTag'] = scoreTag
    scb_hum.index_cols += ['scoreType', 'scoreTag']

    
    # model scores
    cc = [('classifier', classifier), ('M', m)]
    scb_mdl = scbt.keeprows(conditions=cc, comparison='all')

    scoreType = ['model']*scb_mdl.numrows
    scoreTag = [ '%s-%s' % (row['classifier'], row['M']) for _, row in scb_mdl.df.iterrows()]
    scb_mdl.df['scoreType'] = scoreType
    scb_mdl.df['scoreTag'] = scoreTag
    scb_mdl.index_cols += ['scoreType', 'scoreTag']



    scb_out = scb_hum.stack(others=[scb_mdl])

    return scb_out

In [4]:

f_pca = '../../sandbox/ANL-pca-A/pca-test.json'
#f_pcaproj = '../../sandbox/ANL-pca-A/df_pca_prj.csv'
f_features = '../../sandbox/ANL-preprocess-A-train/trial-374/staged-trial-data.json'
f_scores = '../../sandbox/ANL-merged-scores-A/scoreblock-raw-merged.json'


#f_features = '../../sandbox/ANL-preprocess-B-blind/trial-2391/staged-trial-data.json'
#f_scores = '../../sandbox/ANL-merged-scores-B/scoreblock-raw-merged.json'


In [5]:
#%autoreload 2

#edf = rt.EDFData(f_edf)
#pcaprj = pd.read_csv(f_pcaproj, index_col=0)
std = rt.StagedTrialData.from_json(f_features)
pca = rt.PCA.from_json(f_pca)
scb = sb.ScoreBlock.from_json(f_scores)

sbstack = dig_out_scores(s=scb, trial=std.trial, classifier='OVO', m='24h_8ch')

STAGING PARAMETERS SHOULD BE DEPRECATED
-----------------------------------------------------------------------
file               : ../../sandbox/ANL-preprocess-A-train/trial-374/../../../../data/raw_edf/400hz/374.edf
trial              : 374
num datarecords    : 8640
num annotations    : 0
number of signals  : 4
duration [s]       : 86400
startdatetime      : 2019-01-02 9:00:00
epoch duration [s] : 10.0
samples in file    : [34560000 34560000 34560000  3456000]
samples_per_epoch  : [4000. 4000. 4000.  400.]
frequency [Hz]     : [400. 400. 400.  40.]
signal_labels      : ['EEG1', 'EEG2', 'EMG', 'EDFAnnotations']
------------------------------------
  loading signal trace: EEG1
  loading signal trace: EEG2
  loading signal trace: EMG
  loading signal trace: EDFAnnotations
  computing spectrogram: EEG1
  computing spectrogram: EEG2
  computing spectrogram: EMG
  edf load complete: ../../sandbox/ANL-preprocess-A-train/trial-374/../../../../data/raw_edf/400hz/374.edf
---------------------

/home/gbubnis/anaconda2/envs/mouse1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3209: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



In [6]:
%autoreload 2
#matplotlib.rcdefaults()

pp = ep.EEGPlotter(std=std, pca=pca, scores=sbstack)
pp.render(viewEpoch=500)